# Simulación Estocástica
## Tarea N01

Alonso Ogueda Oliva

### Item 1
Escriba una rutina para resolver el sistema de ecuaciones lineales $\mathbf{Ax} = \mathbf{b}$ con:

$$\mathbf{A} = \begin{pmatrix}
1 & 1 & 1 \\
2 & 3 & 1 \\
1 & -1 & -1 \\
\end{pmatrix}
,  \qquad
\mathbf{b} = \begin{pmatrix}
4 \\
9 \\
-1
\end{pmatrix}
$$
usando la **descomposición LU** y **refinamiento iterativo.**

In [1]:
import numpy as np

In [2]:
A = np.array([
    [1., 1., 1.],
    [2., 3., 1.],
    [1., -1., -1.]
])

b = np.array([4, 9, -1])

In [3]:
np.linalg.det(A)

-4.000000000000001

Se define el sistema lineal 
$$ \mathbf{Ax} = \mathbf{LUx} = b$$

Resolviendo el sistema triangular $\mathbf{Lz} = b$ se obtiene que 

In [4]:
def lu_decomposition(A):
    a = A.copy().astype(float)
    n = a.shape[0]
    for k in range(n):
        if a[k, k] == 0:
            raise ValueError
        a[k + 1: n, k] = a[k + 1: n, k] / a[k, k]
        a[k + 1: n, k + 1: n] = a[k + 1: n, k + 1: n] - np.outer(a[k + 1: n, k], a[k, k + 1: n])
    l = np.tril(a, -1)
    u = np.triu(a)
    np.fill_diagonal(l, 1)
    return l, u

In [5]:
L, U = lu_decomposition(A)

In [6]:
np.array_equal(A, L @ U)

True

In [7]:
def back_substitution(T, b):
    T = T.astype(float)
    n = T.shape[0]
    x = np.zeros(n)
    x[n - 1] = b[n - 1] / T[n - 1, n - 1]
    for i in range(n - 2, -1, -1):
        x[i] = (b[i] - np.inner(T[i, i:], x[i:])) / T[i, i]
    return x

In [8]:
def forward_substitution(T, b):
    T = T.astype(float)
    n = T.shape[0]
    x = np.zeros(n)
    x[0] = b[0] / T[0, 0]
    for i in range(1, n):
        x[i] = (b[i] - np.inner(T[i, :i], x[:i])) / T[i, i]
    return x

In [21]:
def lu_solve(A, b):
    L, U = lu_decomposition(A)
    x = back_substitution(U, forward_substitution(L, b))
    return x

In [22]:
x_0 = lu_solve(A, b)
print(x_0)

[1.5  1.75 0.75]


In [33]:
def iterative_refinement(A, b, x_0, k_max=100, tol=10e-6):
    A = A.astype(np.float_)
    b = b.astype(np.float_)
    x_k = x_0.astype(np.float_)
    k = 0
    converge = False
    while not converge:
        r_k = b - A @ x_k
        delta_k = lu_solve(A, r_k)
        x_k += delta_k
        if la.norm(delta_k, np.inf) <= tol * la.norm(x_k, np.inf):
            return x_k
        elif k < k_max:
            k += 1
        else:
            print("Se superó la cantidad de iteraciones.")

In [34]:
iterative_refinement(A, b, x_0, k_max=100, tol=10e-6)

array([1.5 , 1.75, 0.75])

### Item 2
Implementar el operador Sweep usando su lenguaje de programación favorito.

In [78]:
def sweep(A, k):
    if (len(set(A.shape)) != 1) or (A[k, k] == 0):
        raise ValueError
    B = A.copy()
    B[k, k] = 1. / A[k, k]
    B[:, k] = -1 * A[:, k] /  A[k, k]
    B[k, :] = A[k, :] /  A[k, k]
    mask = [i for i in range(A.shape[0]) if i != k]
    B[np.ix_(mask, mask)] = A[np.ix_(mask, mask)] -  np.outer(A[np.ix_(mask, [k])], A[np.ix_([k], mask)]) / A[k, k]
    return B

In [88]:
sA = A.copy()
for i in range(A.shape[0]):
    print(i)
    sA = sweep(sA, i)

0
1
2


In [95]:
a = A.copy()

In [97]:
a[:, k] = -1 * A[:, k] / 100

In [98]:
a

array([[-0.01,  1.  ,  1.  ],
       [-0.02,  3.  ,  1.  ],
       [-0.01, -1.  , -1.  ]])

In [93]:
A @ la.inv(A)

array([[ 1.00000000e+00, -1.11022302e-16,  0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 4.44089210e-16, -1.11022302e-16,  1.00000000e+00]])

In [92]:
A @ sA

array([[ 1.  ,  0.  ,  1.25],
       [ 0.  ,  1.  ,  1.25],
       [ 0.  ,  0.  , -0.25]])

In [94]:
np.outer([1, 2], [3, 4])

array([[3, 4],
       [6, 8]])

In [66]:
k = 0
mask = [x for x in range(A.shape[0]) if x != k]
mask

[1, 2]

In [67]:
A[np.ix_(mask, mask)]

array([[ 3.,  1.],
       [-1., -1.]])

In [69]:
A[np.ix_(mask, [k])]

array([[2.],
       [1.]])

In [70]:
A[np.ix_([k], mask)]

array([[1., 1.]])

In [73]:
A[np.ix_(mask, mask)] -  np.outer(A[np.ix_(mask, [k])], A[np.ix_([k], mask)]) / A[k, k]

array([[ 1., -1.],
       [-2., -2.]])

In [47]:
-1 * A[:, k] /  A[k, k]

array([-0.33333333, -1.        ,  0.33333333])

### Item 3
Pruebe su rutina para obtener la inversa de una matriz usando el operador
Sweep con la siguiente matriz:

$$\mathbf{B} = \begin{pmatrix}
30 & 16 & 46 \\
16 & 10 & 26 \\
46 & 26 & 72 \\
\end{pmatrix}
$$

### Item 4
Verifique que $\mathbf{B}$ es matriz semidefinida positiva usando la factorización Cholesky. ¿En cuál etapa del algoritmo el procedimiento falla?